In [ ]:
import numpy as np
from numpy.random import default_rng

import matplotlib.pyplot as plt
import matplotlib.lines as lines
import matplotlib as mpl
import scipy.spatial.distance as dist

import IBloFunMatch_inter as ibfm
import itertools

output_dir="output" 

### Generate an example with some circles far appart from each other, and some circles mixed.

This should illustrate the potential of partial matchings to detect isolated circles.

In [ ]:
rng = default_rng(22)
X = [] 
for x,y in itertools.product(range(4), range(3)):
    X.append(ibfm.sampled_circle(0.015*(0.8), 0.015*1.2, 20, rng)+[x,y])
# end for 
X = np.vstack(X)
X.shape

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(7,5))
ax.scatter(x=X[:,0], y=X[:,1], c="red", marker="o")
plt.savefig("plots/far_circles/sampled_circles.png")

Next, we take a subset of this point cloud to compute the matchings.

In [ ]:
subset_size = int(X.shape[0]*0.5) # take half of the points
subset_indices = rng.choice(range(len(X)), replace=False, size=subset_size)
S = X[subset_indices]

Compute the matching

In [ ]:
%%capture
Dist_S = dist.squareform(dist.pdist(S))
Dist_X = dist.squareform(dist.pdist(X))
IBloFunMatch_o = ibfm.get_IBloFunMatch_subset(Dist_S, Dist_X, subset_indices, output_dir, max_rad=-1)

In [ ]:
IBloFunMatch_o.keys()

In [ ]:
max(IBloFunMatch_o["matching_strengths_1"])

In [ ]:
# Do plots
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(10,8))
ax[1,0].scatter(S[:,0], S[:,1], c="orange")
ax[1,1].scatter(X[:,0], X[:,1], c="navy")
# Plot Special point 
ax[1,0].set_title("Subset")
ax[1,1].set_title("Dataset")
# Plot first and second edge on dataset
ibfm.plot_matching(IBloFunMatch_o, output_dir, ax[0], fig)
fig.suptitle(f"Far Circles")
plt.savefig(f"plots/far_circles/matching.png")

In [ ]:
# Do plots
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6, 4))
ax.scatter(S[:,0], S[:,1], c="orange")
# Plot Special point 
num_bars = 0
for idx_bar, (rep, bar) in enumerate(zip(IBloFunMatch_o["S_reps_1"], IBloFunMatch_o["S_barcode_1"])):
    if bar[1]<0.4:
        point = X[rep[0]]
        ax.scatter([point[0]], [point[1]], c="blue", s=100, marker="X")
        ax.text(point[0]+0.1, point[1], s=str(idx_bar), c="blue", fontsize=20)
        num_bars+=1
    # end plotting rep
# end rep, bar loop
fig.suptitle(f"Components of reps of {num_bars} early bars")
plt.savefig(f"plots/far_circles/pts_early_bars.png")

In [ ]:
IBloFunMatch_o["matching_strengths_1"]

In [ ]:
max(IBloFunMatch_o["matching_strengths_1"])

### Now, let us repeat the experiment taking out the two central circles, so that there is also an important "global circle"

In [ ]:
rng = default_rng(22)
X = [] 
for x,y in itertools.product(range(4), range(3)):
    if y==1 and (x in [1,2]):
        continue
    X.append(ibfm.sampled_circle(0.015*(0.8), 0.015*1.2, 20, rng)+[x,y])
# end for 
X = np.vstack(X)
X.shape

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(7,5))
ax.scatter(x=X[:,0], y=X[:,1], c="red", marker="o")
plt.savefig("plots/far_circles/sampled_circles.png")

Take the subset and compute the induced matching.

In [ ]:
%%capture
subset_size = int(X.shape[0]*0.4) # take half of the points
subset_indices = rng.choice(range(len(X)), replace=False, size=subset_size)
S = X[subset_indices]
Dist_S = dist.squareform(dist.pdist(S))
Dist_X = dist.squareform(dist.pdist(X))
IBloFunMatch_o = ibfm.get_IBloFunMatch_subset(Dist_S, Dist_X, subset_indices, output_dir)

Plot the induced matching.

In [ ]:
# Do plots
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(10,8))
ax[1,0].scatter(S[:,0], S[:,1], c="orange")
ax[1,1].scatter(X[:,0], X[:,1], c="navy")
# Plot Special point 
ax[1,0].set_title("Subset")
ax[1,1].set_title("Dataset")
# Plot first and second edge on dataset
ibfm.plot_matching(IBloFunMatch_o, output_dir, ax[0], fig)
fig.suptitle(f"Far Circles")
plt.savefig(f"plots/far_circles/matching.png")